In [1]:
import pandas as pd
import spacy
import numpy as np
import nltk
nltk.download('stopwords')
#spacy doesnt work properly in collab, doesn't recognize nlp() function

def list_to_phrase(l):
    temp = ''
    for elm in l:
        temp += elm
        temp += ' '
    return temp[:-1]

def remove_all_blanks(l):
    while '' in l:
        l.remove('')

def get_bigrams(l):
    return list(zip(l[:-1],l[1:]))

def prob_given_parent(bigram, corpus): #calculates probability of child given parent in bigram
    corpus = flatten(corpus)
    numerator = corpus.count(bigram)
    zipped_corpus = list(zip(*corpus))
    denom = zipped_corpus[1].count(bigram[1])
    return numerator/denom

def prob_in_corpus(bigram, corpus): #calculates frequency of bigram in corpus
    corpus = flatten(corpus)
    return corpus.count(bigram)/len(corpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mb546\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = "https://raw.githubusercontent.com/CCrisanti/nlp/master/alexa?token=AFIZDN5X5GJK4DROU52LKRS54XOAQ"
reviews = pd.read_csv(url, sep="\n", header=None)
reviews.head()
stop_words = list(nltk.corpus.stopwords.words('english'))
nlp = spacy.load('en')
#print(reviews.shape)

#creating chunks
reviews = list(reviews.iloc[:,0])
parsed_reviews = [list(nlp(sent).noun_chunks) for sent in reviews]
parsed_reviews = [[str(chunk).split(' ') for chunk in elm] for elm in parsed_reviews]


#removing common pronouns from chunks
for i in range(len(parsed_reviews)):
    for j in range(len(parsed_reviews[i])):
        parsed_reviews[i][j] = [word.lower() for word in parsed_reviews[i][j]]
        for word in stop_words:
            if word.lower() in parsed_reviews[i][j]:
                parsed_reviews[i][j].remove(word)
        parsed_reviews[i][j] =  list_to_phrase(parsed_reviews[i][j]) 
    if '' in parsed_reviews[i]:
        remove_all_blanks(parsed_reviews[i])

#creating bigrams
parsed_reviews_bi = []
for i in range(len(parsed_reviews)):
    parsed_reviews_bi.append(get_bigrams(parsed_reviews[i]))
    
print(parsed_reviews_bi)

[[('great gift', 'anyone'), ('anyone', 'iot devices'), ('iot devices', 'alexa'), ('alexa', 'awesome')], [('gift', 'best buy'), ('best buy', 'echo products'), ('echo products', 'store'), ('store', 'amazon')], [('mother', 'law')], [('designed, good sound', 'everything'), ('everything', 'alexa'), ('alexa', 'hd video'), ('hd video', 'answers'), ('answers', 'associated video'), ('associated video', 'text'), ('text', 'movie trailers'), ('movie trailers', 'amazon video'), ('amazon video', 'demand'), ('demand', 'amazon compatible cameras'), ('amazon compatible cameras', 'voice'), ('voice', 'amazon show owners'), ('amazon show owners', 'alexa app holders')], [('one alexa', 'three echo dots'), ('three echo dots', 'echo show')], [('amazon', 'lot'), ('lot', 'cool stuff')], [('great looking design', 'amazon echo show')], [('product', 'anyone')], [('weeks', 'life')], [('device', 'äò'), ('äò', 'smart home technology'), ('smart home technology', 'äò i‚äôm'), ('äò i‚äôm', 'second one'), ('second one', 

In [3]:


import collections
newlist = [item for items in parsed_reviews_bi for item in items]
#print(newlist)
parents = []
children = []
for pair in newlist:
    parents.append(pair[0])
    children.append(pair[1])

parents_counter = collections.Counter(parents)
children_counter = collections.Counter(children)
pair_counter = collections.Counter(newlist)


#returns a tuple in the form ( p(parent|child), p(child|parent) )
def freq(parent, child):
    parent_count = parents_counter.get(parent)
    child_count = children_counter.get(child)
    pair_count = pair_counter.get((parent,child))

    return (pair_count/parent_count, pair_count/child_count)
    
print(freq('music', 'lyrics'))

print(parents_counter)


(0.06086956521739131, 0.2692307692307692)


Counter({'alexa': 130, 'music': 115, 'echo show': 115, 'screen': 95, 'show': 93, 'echo': 88, 'kitchen': 45, 'amazon': 42, 'family': 42, 'recipes': 40, 'device': 37, 'things': 32, 'house': 32, 'product': 30, 'video': 30, 'videos': 29, 'gift': 28, 'time': 28, 'everything': 25, 'lot': 25, 'dot': 25, 'lyrics': 25, 'wife': 25, 'sound': 25, 'questions': 24, 'one': 23, 'home': 22, 'weather': 21, 'great product': 20, 'news': 20, 'love': 19, 'great addition': 19, 'something': 19, 'amazon music': 18, 'youtube': 18, 'songs': 17, 'us': 17, 'christmas': 16, 'kids': 16, 'thing': 16, 'voice': 15, 'phone': 15, 'amazon show': 14, 'amazon echo show': 14, 'camera': 14, 'echo dot': 14, 'item': 14, 'security cameras': 13, 'amazon echo': 13, 'great device': 12, 'dots': 12, 'best buy': 11, 'speakers': 11, 'son': 11, 'times': 11, 'words': 11, 'work': 11, 'purchase': 11, 'everyone': 10, 'features': 10, 'alexa app': 10, 'hands': 10, 'lights': 10, 'touch': 10, 'video calls': 10, 'ability': 10, 'sale': 9, 'inform

In [4]:
#print(reviews)
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mb546\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
reviews_df = pd.read_csv(url, sep="\n", header=None)

for index, row in reviews_df.iterrows():
  print(row[0])
  polarities = sid.polarity_scores(row[0])
  print(polarities)
  print("\n")
#For now all the polarities are just printed but we could easily store them in a list or a dictionary or whatever

Great Gift for anyone. Very easy to setup. Coexist with all IOT Devices. Alexa is AWESOME!
{'neg': 0.0, 'neu': 0.439, 'pos': 0.561, 'compound': 0.9461}


Super excited to give this as a gift. It's super convenient that Best Buy has Echo products in store instead of having to purchase from Amazon.
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.9584}


We bought this for mother in law, buying another for me.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


Well designed, good sound, has everything Alexa has plus the HD video. Always ready with answers with associated video or text if applicable. Can show movie trailers, can also watch Amazon video. Excellent on-demand security video with Amazon compatible cameras. Voice activated message and/or video calls to Amazon Show owners or Alexa App holders. Highly recommended.
{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'compound': 0.9501}


I have one Alexa and three Echo dots and having Echo Show now is awesome!
{'neg': 0.0, 'neu': 0.7

I was jealous when our son showed us his Amazon Show and some of the things it was capable of doing. Since AOL eliminated Instant Messaging, my wife and I were less able to communicate because our offices are on separate floors. So we got two Shows and two Spots and now not only can intercom easily but use Alexa for music and many other things. We will probably start using some smart home devices which Alexa can also control. It's a fun device which seems to be getting better all the time.
{'neg': 0.02, 'neu': 0.816, 'pos': 0.164, 'compound': 0.9287}


We use this as a monitor for the amazon cloud camera! Works great we tell it to show us the camera ,and boom, there‚Äôs the video feed. Super easy to link our echo plus and the echo dot with the show!!
{'neg': 0.0, 'neu': 0.728, 'pos': 0.272, 'compound': 0.9257}


We use echo almost every day. It is fun to ask questions. The music is great. We check the weather and listen to NPRnews
{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.

Love it took the place off the echo in my bedroom.
{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'compound': 0.6369}


Fantastic product and does exactly as described. Great gift idea!
{'neg': 0.0, 'neu': 0.391, 'pos': 0.609, 'compound': 0.8977}


We really like that when you call someone you can FaceTime with them!
{'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.474}


I bought it as a gift for my parents and they absolutely love it!
{'neg': 0.0, 'neu': 0.565, 'pos': 0.435, 'compound': 0.8264}


We love that even our four year old can tell Alexa to "call papa" or Alexa turn on the kitchen lights!
{'neg': 0.0, 'neu': 0.817, 'pos': 0.183, 'compound': 0.6696}


I find myself saying to my phone, "Alexa, play.........". I wish they made it for my car too.
{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.4019}


Love how the echo show integrates with smart house devices and cameras. I use it to view our cameras, listen to music, make phone calls/video calls and even as a recipe guid

{'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'compound': 0.6486}


At first I was concerned with the funny looking angle I saw on the pictures. The item was not displayed at the local store so I decided to take a chance. As an owner of the Echo Dot i figured it would work in the same manner and it does. I love that it works with the Arlo Pro. The screen is great and I soon figured out the reason for the funny angle of the item. The angled screen works great Only setback or recommendation would be to add a auxiliary output at the rear. Also you can watch prime movies, YouTube videos etc.
{'neg': 0.0, 'neu': 0.818, 'pos': 0.182, 'compound': 0.9648}


I bought the white version and have it in the kitchen. I use it for EVERYTHING, music, video, phone, security cams, news, weather. It's the perfect kitchen assistant.
{'neg': 0.0, 'neu': 0.797, 'pos': 0.203, 'compound': 0.7269}


This Alexa is the best one out of all them due to the screen feature
{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compou

I love that you and now see your answer on a screen with the Alexa Echo Show!!!
{'neg': 0.0, 'neu': 0.734, 'pos': 0.266, 'compound': 0.7249}


It was a bit more involved getting set up but thereafter the function is simply talking to Alexia. The plug adapter to turn on and off lights TV is useful and certainly worth having.
{'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'compound': 0.8519}


Adds some nice tricks to the regular Echo lineup. Video calls to other Show owners works well. Speakers sound good.
{'neg': 0.059, 'neu': 0.632, 'pos': 0.308, 'compound': 0.743}


THE ECHO SHOW WORKS WELL, BUT AT THIS TIME IT CANNOT CONNECT TO YOUTUBE. MAYBE WHEN AMAZON WAKES UP THE CONNECTION WILL BE RETURNED.
{'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'compound': 0.3818}


It's a lot of fun. To fully enjoy it, we had to get Amazon music unlimited. We bought four of them, so we can keep in touch with family. Alexa is very entertaining!
{'neg': 0.0, 'neu': 0.7, 'pos': 0.3, 'compound': 0.8996}


I seen one 

...

The Echo Show is a great improvement from the original Echo device. It's great being able to "see" what is going on!!
{'neg': 0.0, 'neu': 0.547, 'pos': 0.453, 'compound': 0.9335}


The Echo Dot is a great personal assistant, and right now the price is fantastic. It can give you the weather forecast, traffic reports, daily news brief, play music from Pandora or other steaming radio stations, and answer questions, all in a very small package.. .
{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'compound': 0.8779}


Bought as a gift for my wife, she loves it and the screen is very handy for recipes and such.
{'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compound': 0.765}


This thing is cool, you can do a lot of neat things with the show, hope they add more functionality to Alexa though.
{'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.802}


We have echos throughout the whole house and have this in the kitchen. Controls lights , you can FaceTime , my wife uses it for recipes and it shows the vid

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





Absolutely love this. We got the lights and they have worked so great.
{'neg': 0.0, 'neu': 0.531, 'pos': 0.469, 'compound': 0.8942}


Great Smart speaker for my 90 year old mother in law.
{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compound': 0.7783}


I‚Äôm an amazon prime member and the show is awsum I use it to communicate with the son who lives in Seattle. You can watch amazon pirime tv,play amazon music. Text, play games read the lyrics while the song is playing. I love it
{'neg': 0.0, 'neu': 0.727, 'pos': 0.273, 'compound': 0.8885}


We loved the Echo Dot, but the Show is exponentially better. Especially love to see the words of the songs being played, and views of places all over the world we want to see. Big difference in cost between dot and show, but worth it!
{'neg': 0.0, 'neu': 0.664, 'pos': 0.336, 'compound': 0.9624}


Love the new echo show. has so many more features that my echo dot.
{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'compound': 0.6369}


I had been looking for the 

I picked up the show on a Black Friday deal. I probably would never have paid full price. I am glad I have the show, it is actually quite useful. The sound quality is also better than the standard echo. At the moment, my only complaints are the lack of YouTube support and the price.. It will be interesting to see what features Amazon will add to the show in the future.
{'neg': 0.062, 'neu': 0.736, 'pos': 0.202, 'compound': 0.8805}


This is an awesome product. This is the first Echo product I owned and I purchased 2 to get $100 off. I was going to split with a friend but like this so much that I decided to keep both. I've being singing along with the stream song with lyrics while doing work in the kitchen and house and watching youtube videos. The videos on Youtube takes a more effort to find the ones I want but I think this will improve over time.
{'neg': 0.0, 'neu': 0.841, 'pos': 0.159, 'compound': 0.9042}


I love all of my Amazon echo devices. Right now it doesn't allow YouTube, bu

I bought the white version and have it in the kitchen. I use it for EVERYTHING, music, video, phone, security cams, news, weather. It's the perfect kitchen assistant.
{'neg': 0.0, 'neu': 0.797, 'pos': 0.203, 'compound': 0.7269}


BUY IT !The video adds great value to the songs, news, etc.Also, accessing videos such as Netflix adds greatly.
{'neg': 0.0, 'neu': 0.715, 'pos': 0.285, 'compound': 0.7777}


The echo show feels useless after a few hours, especially without the ability to watch YouTube.
{'neg': 0.279, 'neu': 0.721, 'pos': 0.0, 'compound': -0.6156}


This device is especially useful if you know others who own it. It offers the same features as the standard Echo, along with an expanded feature set as well.
{'neg': 0.0, 'neu': 0.836, 'pos': 0.164, 'compound': 0.6478}


My entire family loves this product. We ask her questions and go to sleep with her playing music. You feel like you have company anytime you want
{'neg': 0.0, 'neu': 0.712, 'pos': 0.288, 'compound': 0.8074}


This 

In [6]:
class node:
    def __init__(self, value):
        self.children = set()
        self.value = value
        
print(parents_counter.most_common(1)[0][0])
root = node(parents_counter.most_common(1)[0][0])

currentNode = root
for tup in newlist:
    #print(tup)
    if tup[0] == currentNode.value:
        currentNode.children.add(node(tup[1]))

for i in root.children:
    currentNode = i
    for tup in newlist:
        #print(tup)
        if tup[0] == currentNode.value:
            currentNode.children.add(node(tup[1]))
    


    

alexa


In [7]:
from anytree import Node, RenderTree

root = Node(parents_counter.most_common(1)[0][0])

currentNode = root
for tup in newlist:
    #print(tup)
    if tup[0] == currentNode.name:
        exists = False
        for i in currentNode.children:
            if i.name == tup[1]:
                exists = True
        if(not exists):
            Node(tup[1], parent=currentNode)


#for pre, fill, node in RenderTree(root):
#    print("%s%s" % (pre, node.name))

for i in root.children:
    currentNode = i
    for tup in newlist:
        #print(tup)
        if tup[0] == currentNode.name:
            exists = False
            for k in currentNode.children:
                if k.name == tup[1]:
                    exists = True
            if(not exists):
                Node(tup[1], parent=currentNode)
            
                        
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))
    

alexa
├── awesome
│   └── love
├── hd video
│   └── answers
├── commands
│   ├── command
│   ├── questions
│   ├── night
│   ├── cameras
│   ├── excellent product
│   ├── long learning curve
│   ├── room
│   └── tic
├── love
│   ├── news briefings
│   ├── music
│   ├── echo
│   ├── things
│   ├── alexa
│   ├── christmas music
│   ├── much talk & c peeps
│   ├── device
│   ├── playing music
│   ├── people
│   ├── face
│   ├── daily news briefing
│   └── kitchen
├── music
│   ├── events
│   ├── video conferencing
│   ├── amazon music
│   ├── lyrics
│   ├── many things
│   ├── weather
│   ├── alexa
│   ├── family
│   ├── lights
│   ├── voice command
│   ├── every one
│   ├── new recipes
│   ├── phone calls/video calls
│   ├── echo.2-
│   ├── 6- 3) buttons
│   ├── sports
│   ├── video
│   ├── words
│   ├── decent quality sound
│   ├── video chat
│   ├── demand
│   ├── itunes
│   ├── videos
│   ├── set timers
│   ├── general information
│   ├── news briefing
│   ├── security cameras
│   ├──

│   ├── house
│   ├── project
│   ├── bing
│   ├── top
│   ├── show
│   ├── echo
│   ├── volume
│   └── home
├── gadget
│   └── kitchen
├── show
│   ├── video screen
│   ├── recipes
│   ├── movies
│   ├── show
│   ├── calls
│   ├── volume
│   ├── picture
│   ├── center
│   ├── walls
│   ├── good picture
│   ├── great job
│   ├── outside
│   ├── convenience
│   ├── smart lights
│   ├── priciest piece
│   ├── video feature
│   ├── kitchen
│   ├── one alexa-enabled machine
│   ├── 10 year old grandson
│   ├── essentially visual phone calls
│   ├── good device
│   ├── internet cameras
│   ├── setup
│   ├── touch screen
│   ├── camera
│   ├── couple
│   ├── mini tips
│   ├── screen
│   ├── best sounding
│   ├── music
│   ├── awsum
│   ├── spot
│   ├── echo plus
│   ├── amazon's shows
│   ├── cameras
│   ├── must
│   ├── ring doorbell
│   ├── us
│   ├── echo
│   ├── alexa
│   ├── amazon prime account
│   ├── functionality
│   ├── words
│   ├── products
│   ├── parents
│   ├── living room
│  

│   ├── children's families
│   ├── home
│   ├── network
│   ├── show
│   ├── date news
│   ├── recipes
│   └── timers
├── dots
│   ├── house
│   ├── couple shows
│   ├── one
│   ├── show
│   ├── echos
│   ├── video calls
│   ├── bedrooms
│   ├── almost every room
│   ├── everyone
│   ├── echo
│   └── cameras
├── way
│   ├── device
│   ├── beer
│   ├── music
│   ├── favorites
│   ├── devices
│   └── camera
├── answer
│   ├── screen
│   ├── questions
│   ├── everything
│   └── news
├── home automation products
│   └── show
├── family member
│   └── life
├── customer service
│   └── amazon
├── play
│   ├── home
│   └── brackets
├── booming sound
│   └── room
├── climbing
│   └── closet
├── tablet
│   ├── voice
│   ├── ipod
│   ├── tv
│   ├── cell phone
│   ├── smart phone device
│   └── nice speaker
├── pretty much anything
├── strong internet search engine
├── shopping list
│   ├── visual confirmation
│   ├── lot
│   ├── talk
│   ├── timer
│   └── touch
├── hand
│   ├── videos
│   └── h

In [8]:
dummy_path = ['echo show','alexa','great speakers']
def happy_children(path):
    node_pointer = root
    node_pointer.name = path[0]
    for i in range(len(path)-1):
        for child in node_pointer.children:
            if child.name == path[i+1]:
                node_pointer = child
    abc = node_pointer.parent
    our_list = list(abc.children)
    our_list = [i for i in our_list if i!=node_pointer]
    abc.children = tuple(our_list)

happy_children(dummy_path)

    


In [9]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

echo show
├── awesome
│   └── love
├── hd video
│   └── answers
├── commands
│   ├── command
│   ├── questions
│   ├── night
│   ├── cameras
│   ├── excellent product
│   ├── long learning curve
│   ├── room
│   └── tic
├── love
│   ├── news briefings
│   ├── music
│   ├── echo
│   ├── things
│   ├── alexa
│   ├── christmas music
│   ├── much talk & c peeps
│   ├── device
│   ├── playing music
│   ├── people
│   ├── face
│   ├── daily news briefing
│   └── kitchen
├── music
│   ├── events
│   ├── video conferencing
│   ├── amazon music
│   ├── lyrics
│   ├── many things
│   ├── weather
│   ├── alexa
│   ├── family
│   ├── lights
│   ├── voice command
│   ├── every one
│   ├── new recipes
│   ├── phone calls/video calls
│   ├── echo.2-
│   ├── 6- 3) buttons
│   ├── sports
│   ├── video
│   ├── words
│   ├── decent quality sound
│   ├── video chat
│   ├── demand
│   ├── itunes
│   ├── videos
│   ├── set timers
│   ├── general information
│   ├── news briefing
│   ├── security cameras
│  

│   ├── bing
│   ├── top
│   ├── show
│   ├── echo
│   ├── volume
│   └── home
├── gadget
│   └── kitchen
├── show
│   ├── video screen
│   ├── recipes
│   ├── movies
│   ├── show
│   ├── calls
│   ├── volume
│   ├── picture
│   ├── center
│   ├── walls
│   ├── good picture
│   ├── great job
│   ├── outside
│   ├── convenience
│   ├── smart lights
│   ├── priciest piece
│   ├── video feature
│   ├── kitchen
│   ├── one alexa-enabled machine
│   ├── 10 year old grandson
│   ├── essentially visual phone calls
│   ├── good device
│   ├── internet cameras
│   ├── setup
│   ├── touch screen
│   ├── camera
│   ├── couple
│   ├── mini tips
│   ├── screen
│   ├── best sounding
│   ├── music
│   ├── awsum
│   ├── spot
│   ├── echo plus
│   ├── amazon's shows
│   ├── cameras
│   ├── must
│   ├── ring doorbell
│   ├── us
│   ├── echo
│   ├── alexa
│   ├── amazon prime account
│   ├── functionality
│   ├── words
│   ├── products
│   ├── parents
│   ├── living room
│   ├── great choice
│   ├── new 

│   ├── network
│   ├── show
│   ├── date news
│   ├── recipes
│   └── timers
├── dots
│   ├── house
│   ├── couple shows
│   ├── one
│   ├── show
│   ├── echos
│   ├── video calls
│   ├── bedrooms
│   ├── almost every room
│   ├── everyone
│   ├── echo
│   └── cameras
├── way
│   ├── device
│   ├── beer
│   ├── music
│   ├── favorites
│   ├── devices
│   └── camera
├── answer
│   ├── screen
│   ├── questions
│   ├── everything
│   └── news
├── home automation products
│   └── show
├── family member
│   └── life
├── customer service
│   └── amazon
├── play
│   ├── home
│   └── brackets
├── booming sound
│   └── room
├── climbing
│   └── closet
├── tablet
│   ├── voice
│   ├── ipod
│   ├── tv
│   ├── cell phone
│   ├── smart phone device
│   └── nice speaker
├── pretty much anything
├── strong internet search engine
├── shopping list
│   ├── visual confirmation
│   ├── lot
│   ├── talk
│   ├── timer
│   └── touch
├── hand
│   ├── videos
│   └── help
├── lyrics
│   ├── music
│   ├── song